# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [40]:
import requests
import pyowm
import json

api_key = 'a40d1f6f7bde55793059a76a8dd8062c'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [41]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key

}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 287.84,
  'feels_like': 287.01,
  'temp_min': 285.71,
  'temp_max': 288.76,
  'pressure': 1023,
  'humidity': 63,
  'sea_level': 1023,
  'grnd_level': 1017},
 'visibility': 10000,
 'wind': {'speed': 1.34, 'deg': 53, 'gust': 2.68},
 'clouds': {'all': 79},
 'dt': 1745497757,
 'sys': {'type': 2,
  'id': 268730,
  'country': 'GB',
  'sunrise': 1745469938,
  'sunset': 1745521877},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [45]:
#your code here
from pyowm.owm import OWM
from pyowm.utils.config import get_default_config

config_dict = get_default_config()
config_dict['language'] = 'ru'  # your language here, eg. French
owm = OWM(api_key, config_dict)
mgr = owm.weather_manager()
observation = mgr.weather_at_place('Berlin, Germany')
print(observation.weather.detailed_status)


ясно


## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [25]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°K, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 286.95°F, Humidity: 64%, Wind Speed: 3.09 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [35]:
#your code here
observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
weather.status           # short version of status (eg. 'Rain')
display(weather.detailed_status)
temp_dict_celsius = weather.temperature('celsius')
display(temp_dict_celsius)

'облачно с прояснениями'

{'temp': 14.37,
 'temp_max': 15.1,
 'temp_min': 12.06,
 'feels_like': 13.51,
 'temp_kf': None}

## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [38]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

display(geocode_response)

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

{'coord': {'lon': -74.006, 'lat': 40.7143},
 'weather': [{'id': 711,
   'main': 'Smoke',
   'description': 'smoke',
   'icon': '50d'}],
 'base': 'stations',
 'main': {'temp': 289.15,
  'feels_like': 288.16,
  'temp_min': 287.01,
  'temp_max': 290.92,
  'pressure': 1025,
  'humidity': 52,
  'sea_level': 1025,
  'grnd_level': 1025},
 'visibility': 10000,
 'wind': {'speed': 3.09, 'deg': 80},
 'clouds': {'all': 0},
 'dt': 1745497742,
 'sys': {'type': 1,
  'id': 4610,
  'country': 'US',
  'sunrise': 1745489012,
  'sunset': 1745538260},
 'timezone': -14400,
 'id': 5128581,
 'name': 'New York',
 'cod': 200}

Then, request the weather forecast for that latitude/longitude:

In [37]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-04-24 15:00:00, Temp: 290.44°C
Time: 2025-04-24 18:00:00, Temp: 295.27°C
Time: 2025-04-24 21:00:00, Temp: 297.69°C
Time: 2025-04-25 00:00:00, Temp: 293.31°C
Time: 2025-04-25 03:00:00, Temp: 289.3°C
Time: 2025-04-25 06:00:00, Temp: 290.02°C
Time: 2025-04-25 09:00:00, Temp: 288.46°C
Time: 2025-04-25 12:00:00, Temp: 288.98°C
Time: 2025-04-25 15:00:00, Temp: 293.56°C
Time: 2025-04-25 18:00:00, Temp: 293.35°C
Time: 2025-04-25 21:00:00, Temp: 291.68°C
Time: 2025-04-26 00:00:00, Temp: 290.58°C
Time: 2025-04-26 03:00:00, Temp: 288.43°C
Time: 2025-04-26 06:00:00, Temp: 288.25°C
Time: 2025-04-26 09:00:00, Temp: 287.84°C
Time: 2025-04-26 12:00:00, Temp: 288.97°C
Time: 2025-04-26 15:00:00, Temp: 290.59°C
Time: 2025-04-26 18:00:00, Temp: 294.25°C
Time: 2025-04-26 21:00:00, Temp: 291.5°C
Time: 2025-04-27 00:00:00, Temp: 287.17°C
Time: 2025-04-27 03:00:00, Temp: 284.06°C
Time: 2025-04-27 06:00:00, Temp: 282.73°C
Time: 2025-04-27 09:00:00, Temp: 282.71°C
Time: 2025-04-27 12:00:00, Temp: 283

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [70]:
#your code here
from pyowm.utils import timestamps
import pprint

mgr = owm.weather_manager()
h3_forecast = mgr.forecast_at_place('Berlin,DE', '3h')
tomorrow_at_five = timestamps.tomorrow(17, 0)                      # datetime object for tomorrow at 5 PM
weather = h3_forecast.get_weather_at(tomorrow_at_five) 
weather_data = weather.to_dict()
weather_data


{'reference_time': 1745604000,
 'sunset_time': None,
 'sunrise_time': None,
 'clouds': 90,
 'rain': {},
 'snow': {},
 'wind': {'speed': 4.92, 'deg': 20, 'gust': 7.79},
 'humidity': 46,
 'pressure': {'press': 1020, 'sea_level': 1020},
 'temperature': {'temp': 285.44,
  'temp_kf': 0,
  'temp_max': 285.44,
  'temp_min': 285.44,
  'feels_like': 283.93},
 'status': 'Clouds',
 'detailed_status': 'пасмурно',
 'weather_code': 804,
 'weather_icon_name': '04d',
 'visibility_distance': 10000,
 'dewpoint': None,
 'humidex': None,
 'heat_index': None,
 'utc_offset': None,
 'uvi': None,
 'precipitation_probability': 0}

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [ ]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [74]:
#your code here
observation = mgr.weather_at_place('Dubai')  # the observation object is a box containing a weather object
weather = observation.weather
weather.status           # short version of status (eg. 'Rain')

temp_dict_celsius = weather.temperature('celsius')
temp_dict_farenheit = weather.temperature('fahrenheit')
print(f"temp {temp_dict_celsius['temp']}°C, Fahrenheit: {temp_dict_farenheit['temp']}°F")

temp 30.96°C, Fahrenheit: 87.73°F
